In [1]:
import numpy as np
import numpy as np
from scipy.optimize import minimize
from scipy.stats import poisson

In [2]:
def expected_binned_events(mu, phi, N, delta_t, n, d):
    t_start = (n-1) * delta_t
    t_end = n * delta_t
    t = np.linspace(t_start, t_end, 100)  # Discretize time
    
    # First term: integral of mu
    first_term = np.trapz(mu(t), t)
    
    # Second term: sum of integrals
    second_term = 0
    for j in range(d):
        inner_integral = np.array([np.trapz(phi(t-u) * N[j](u), u[:i+1]) for i, _ in enumerate(t)])
        second_term += np.trapz(inner_integral, t)
    
    return first_term + second_term

In [3]:
def kernel_integral_approximation(phi, X, delta, t_min, t_max):
    K = len(X)
    result = 0
    for k in range(K):
        u = np.linspace(t_min[k], t_max[k], 100)
        result += (X[k] / delta[k]) * np.trapz(phi(u), u)
    return result


In [4]:
def expected_events_approximation(mu, phi, X, delta, t_min, t_max, Delta, d):
    first_term = Delta * mu
    
    second_term = 0
    for j in range(d):
        for k in range(len(X)):
            u = np.linspace(t_min[k], t_max[k], 100)
            second_term += (X[k] / delta[k]) * np.trapz(phi(u), u)
    second_term *= Delta
    
    return first_term + second_term

In [5]:
def expected_in_spread_events(f, lambda_e, s, beta, Delta, n):
    t_start = (n-1) * Delta
    t_end = n * Delta
    t = np.linspace(t_start, t_end, 100)  # Discretize time
    
    integrand = (s-1)**beta * f(t) * lambda_e(t)
    return np.trapz(integrand, t)

def expected_in_spread_events_ratio(mu, phi, N, f, s, beta, Delta, n, d):
    numerator = expected_in_spread_events(f, lambda t: mu(t), s, beta, Delta, n)
    
    denominator = f(Delta*n) * np.mean((s-1)**beta)
    
    t_start = (n-1) * Delta
    t_end = n * Delta
    t = np.linspace(t_start, t_end, 100)  # Discretize time
    
    first_term = np.trapz(mu(t), t)
    
    second_term = 0
    for j in range(d):
        inner_integral = np.array([np.trapz(phi(t-u) * N[j](u), u[:i+1]) for i, _ in enumerate(t)])
        second_term += np.trapz(inner_integral, t)
    
    return (numerator / denominator, first_term + second_term)


In [6]:
def kernel_integral_bound(phi, t_min, t_max, f, Q_t):
    K = len(t_min)
    integral_approx = sum(np.trapz(phi(np.linspace(t_min[k], t_max[k], 100)), 
                                   np.linspace(t_min[k], t_max[k], 100)) 
                          for k in range(K))
    
    upper_bound = 1 / np.max(f(Q_t))
    
    return integral_approx, upper_bound

In [7]:
def in_spread_kernel_integral_bound(phi, t_min, t_max, f, Q_t, s, beta):
    K = len(t_min)
    integral_approx = sum(np.trapz(phi(np.linspace(t_min[k], t_max[k], 100)), 
                                   np.linspace(t_min[k], t_max[k], 100)) 
                          for k in range(K))
    
    s_factor = np.mean((s-1)**beta)
    upper_bound = 1 / (s_factor * np.max(f(Q_t)))
    
    return integral_approx, upper_bound

In [8]:
def estimate_exogenous_intensity(M_hat, Lambda_hat):
    I = np.eye(M_hat.shape[0])  # Identity matrix
    mu_hat = np.dot(I - M_hat, Lambda_hat)
    return mu_hat

In [9]:
class LOBState:
    def __init__(self, ask_prices, bid_prices, ask_volumes, bid_volumes):
        self.ask_prices = ask_prices
        self.bid_prices = bid_prices
        self.ask_volumes = ask_volumes
        self.bid_volumes = bid_volumes

    def update_is_order(self, side, price, volume):
        if side == 'ask':
            idx = np.searchsorted(self.ask_prices, price)
            self.ask_prices = np.insert(self.ask_prices, idx, price)
            self.ask_volumes = np.insert(self.ask_volumes, idx, volume)
        else:
            idx = np.searchsorted(self.bid_prices, price)
            self.bid_prices = np.insert(self.bid_prices, idx, price)
            self.bid_volumes = np.insert(self.bid_volumes, idx, volume)

    def update_qd_order(self, side, volume):
        if side == 'ask':
            self.ask_volumes[0] -= volume
            if self.ask_volumes[0] <= 0:
                self.ask_prices = self.ask_prices[1:]
                self.ask_volumes = self.ask_volumes[1:]
        else:
            self.bid_volumes[0] -= volume
            if self.bid_volumes[0] <= 0:
                self.bid_prices = self.bid_prices[1:]
                self.bid_volumes = self.bid_volumes[1:]

In [10]:
class CompoundHawkesProcess:
    def __init__(self, d, mu, phi, theta):
        self.d = d
        self.mu = mu
        self.phi = phi
        self.theta = theta
        self.event_times = [[] for _ in range(d)]

    def intensity(self, t, i):
        lambda_i = self.mu[i]
        for j in range(self.d):
            for tj in self.event_times[j]:
                if tj < t:
                    lambda_i += self.phi[j][i](t - tj)
        return lambda_i

    def simulate(self, T):
        t = 0
        while t < T:
            lambda_total = sum(self.intensity(t, i) for i in range(self.d))
            t += np.random.exponential(1 / lambda_total)
            if t > T:
                break
            u = np.random.random()
            s = 0
            for i in range(self.d):
                s += self.intensity(t, i) / lambda_total
                if u <= s:
                    self.event_times[i].append(t)
                    break

In [11]:
class LOBHawkesModel:
    def __init__(self, initial_state, hawkes_process):
        self.state = initial_state
        self.hawkes = hawkes_process

    def simulate(self, T):
        self.hawkes.simulate(T)
        for i, times in enumerate(self.hawkes.event_times):
            for t in times:
                if i < 6:  # Limit order events
                    side = 'ask' if i % 2 == 0 else 'bid'
                    price = self.state.ask_prices[0] if side == 'ask' else self.state.bid_prices[0]
                    volume = np.random.poisson(self.hawkes.theta[i])
                    self.state.update_is_order(side, price, volume)
                else:  # Market order events
                    side = 'ask' if i % 2 == 0 else 'bid'
                    volume = np.random.poisson(self.hawkes.theta[i])
                    self.state.update_qd_order(side, volume)

In [12]:
def calibrate_hawkes(data, T, d):
    def objective(params):
        mu = params[:d]
        phi_params = params[d:d*(d+1)].reshape(d, d)
        theta = params[d*(d+1):]

        def phi(t, i, j):
            return phi_params[i, j] * np.exp(-phi_params[i, j] * t)

        hawkes = CompoundHawkesProcess(d, mu, phi, theta)
        hawkes.simulate(T)

        log_likelihood = 0
        for i in range(d):
            log_likelihood += sum(np.log(hawkes.intensity(t, i)) for t in data[i])
            log_likelihood -= np.trapz([hawkes.intensity(t, i) for t in np.linspace(0, T, 1000)], np.linspace(0, T, 1000))

        return -log_likelihood

    initial_params = np.random.rand(d * (d + 2))  # mu, phi_params, theta
    result = minimize(objective, initial_params, method='L-BFGS-B', bounds=[(0, None)] * len(initial_params))
    return result.x


In [ ]:
# Initialize LOB state
initial_state = LOBState(
    ask_prices=np.array([100.0, 100.1, 100.2]),
    bid_prices=np.array([99.9, 99.8, 99.7]),
    ask_volumes=np.array([1000, 2000, 3000]),
    bid_volumes=np.array([1500, 2500, 3500])
)

# Initialize Hawkes process parameters
d = 12  # 6 for limit orders, 6 for market orders
mu = np.random.rand(d)
phi = [[lambda t: np.exp(-t) for _ in range(d)] for _ in range(d)]
theta = np.random.rand(d)

hawkes_process = CompoundHawkesProcess(d, mu, phi, theta)

# Create LOB Hawkes model
model = LOBHawkesModel(initial_state, hawkes_process)

# Simulate the model
T = 100  # Simulation time
model.simulate(T)

# Prepare data for calibration
data = model.hawkes.event_times

# Calibrate the model
calibrated_params = calibrate_hawkes(data, T, d)

# Print results
print("Calibrated parameters:")
print("mu:", calibrated_params[:d])
print("phi_params:", calibrated_params[d:d*(d+1)].reshape(d, d))
print("theta:", calibrated_params[d*(d+1):])